## Finetuning TII's Falcon-7B Model on Intel Max Series GPUs 🚀

In this notebook, we will be fine-tuning TII's Falcon-7B foundational model. 💪
___

#### Step 1: Setting Up the Environment 🛠️

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import sys
import site
import os

# Install the required packages
!{sys.executable} -m pip install --upgrade  "transformers>=4.38.*"
!{sys.executable} -m pip install --upgrade  "datasets>=2.18.*"
!{sys.executable} -m pip install --upgrade "wandb>=0.16.*"
!{sys.executable} -m pip install --upgrade "trl>=0.7.11"
!{sys.executable} -m pip install --upgrade "peft>=0.9.0"
!{sys.executable} -m pip install --upgrade "accelerate>=0.28.*"

# Get the site-packages directory
site_packages_dir = site.getsitepackages()[0]

# add the site pkg directory where these pkgs are insalled to the top of sys.path
if not os.access(site_packages_dir, os.W_OK):
    user_site_packages_dir = site.getusersitepackages()
    if user_site_packages_dir in sys.path:
        sys.path.remove(user_site_packages_dir)
    sys.path.insert(0, user_site_packages_dir)
else:
    if site_packages_dir in sys.path:
        sys.path.remove(site_packages_dir)
    sys.path.insert(0, site_packages_dir)

We'll now make sure to optimize our environment for the Intel GPU by setting the appropriate environment variables and configuring the number of cores and threads. This will ensure we get the best performance out of our hardware! ⚡

In [3]:
import warnings
warnings.filterwarnings("ignore")

import os
import psutil

num_physical_cores = psutil.cpu_count(logical=False)
num_cores_per_socket = num_physical_cores // 2

os.environ["TOKENIZERS_PARALLELISM"] = "0"
#HF_TOKEN = os.environ["HF_TOKEN"]

# Set the LD_PRELOAD environment variable
ld_preload = os.environ.get("LD_PRELOAD", "")
conda_prefix = os.environ.get("CONDA_PREFIX", "")
# Improve memory allocation performance, if tcmalloc is not available, please comment this line out
os.environ["LD_PRELOAD"] = f"{ld_preload}:{conda_prefix}/lib/libtcmalloc.so"
# Reduce the overhead of submitting commands to the GPU
os.environ["SYCL_PI_LEVEL_ZERO_USE_IMMEDIATE_COMMANDLISTS"] = "1"
# reducing memory accesses by fusing SDP ops
os.environ["ENABLE_SDP_FUSION"] = "1"
# set openMP threads to number of physical cores
os.environ["OMP_NUM_THREADS"] = str(num_physical_cores)
# Set the thread affinity policy
os.environ["OMP_PROC_BIND"] = "close"
# Set the places for thread pinning
os.environ["OMP_PLACES"] = "cores"

print(f"Number of physical cores: {num_physical_cores}")
print(f"Number of cores per socket: {num_cores_per_socket}")
print(f"OpenMP environment variables:")
print(f"  - OMP_NUM_THREADS: {os.environ['OMP_NUM_THREADS']}")
print(f"  - OMP_PROC_BIND: {os.environ['OMP_PROC_BIND']}")
print(f"  - OMP_PLACES: {os.environ['OMP_PLACES']}")

___
#### Step 2: Initializing the XPU and monitoring GPU memory in realtime 🎮

In [4]:
import asyncio
import threading
import torch
from IPython.display import display, HTML

import torch
import intel_extension_for_pytorch as ipex

if torch.xpu.is_available():
    torch.xpu.empty_cache()
    
    def get_memory_usage():
        memory_reserved = round(torch.xpu.memory_reserved() / 1024**3, 3)
        memory_allocated = round(torch.xpu.memory_allocated() / 1024**3, 3)
        max_memory_reserved = round(torch.xpu.max_memory_reserved() / 1024**3, 3)
        max_memory_allocated = round(torch.xpu.max_memory_allocated() / 1024**3, 3)
        return memory_reserved, memory_allocated, max_memory_reserved, max_memory_allocated
   
    def print_memory_usage():
        device_name = torch.xpu.get_device_name()
        print(f"XPU Name: {device_name}")
        memory_reserved, memory_allocated, max_memory_reserved, max_memory_allocated = get_memory_usage()
        memory_usage_text = f"XPU Memory: Reserved={memory_reserved} GB, Allocated={memory_allocated} GB, Max Reserved={max_memory_reserved} GB, Max Allocated={max_memory_allocated} GB"
        print(f"\r{memory_usage_text}", end="", flush=True)
    
    async def display_memory_usage(output):
        device_name = torch.xpu.get_device_name()
        output.update(HTML(f"<p>XPU Name: {device_name}</p>"))
        while True:
            memory_reserved, memory_allocated, max_memory_reserved, max_memory_allocated = get_memory_usage()
            memory_usage_text = f"XPU ({device_name}) :: Memory: Reserved={memory_reserved} GB, Allocated={memory_allocated} GB, Max Reserved={max_memory_reserved} GB, Max Allocated={max_memory_allocated} GB"
            output.update(HTML(f"<p>{memory_usage_text}</p>"))
            await asyncio.sleep(5)
    
    def start_memory_monitor(output):
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        loop.create_task(display_memory_usage(output))
        thread = threading.Thread(target=loop.run_forever)
        thread.start()    
    output = display(display_id=True)
    start_memory_monitor(output)
else:
    print("XPU device not available.")

___
#### Step 3: Loading the Model 🤖

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

USE_CPU = False
device = "xpu:0" if torch.xpu.is_available() else "cpu"
if USE_CPU:
    device = "cpu"
print(f"using device: {device}")

model_id = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
# Set padding side to the right to ensure proper attention masking during fine-tuning
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
# Disable caching mechanism to reduce memory usage during fine-tuning
model.config.use_cache = False
# Configure the model's pre-training tensor parallelism degree to match the fine-tuning setup
model.config.pretraining_tp = 1 

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

___
#### Step 4: Configuring the LoRA Settings 🎛️

In [13]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

___
#### Step 5: Testing the Model 🧪

In [14]:
def generate_response(model, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)    
    outputs = model.generate(input_ids, max_new_tokens=100,
                             eos_token_id=tokenizer.eos_token_id)    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def test_model(model, test_inputs):
    """quickly test the model using queries."""
    for input_text in test_inputs:
        print("__"*25)
        generated_response = generate_response(model, input_text)
        print(f"{input_text}")
        print(f"Generated Answer: {generated_response}\n")
        print("__"*25)

test_inputs = [
    "What are the main differences between a vegetarian and a vegan diet?",
    "What are some effective strategies for managing stress and anxiety?",
    "Can you explain the concept of blockchain technology in simple terms?",
    "What are the key factors that influence the price of crude oil in global markets?",
    "When did Virgin Australia start operating?"
]

print("Testing the model before fine-tuning:")
test_model(model, test_inputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
The atte

___
#### Step 6: Preparing the Dataset 📊

In [15]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"
dataset = load_dataset(dataset_name, split="train")

print(f"Instruction is: {dataset[0]['instruction']}")
print(f"Response is: {dataset[0]['response']}")

# Filter only question Answers
categories_to_keep = ["close_qa", "open_qa", "general_qa"]
filtered_dataset = dataset.filter(lambda example: example['category'] in categories_to_keep)

print(f"Number of examples in the filtered dataset: {len(filtered_dataset)}")
print(f"Categories in the filtered dataset: {filtered_dataset['category'][:10]}")

# Remove unwanted fields from the filtered dataset
dataset = filtered_dataset.remove_columns(["context", "category"])
print(f"Number of examples in the dataset: {len(dataset)}")
print(f"Fields in the dataset: {list(dataset.features.keys())}")

2024-06-02 20:12:08,081 - datasets - INFO - PyTorch version 2.1.0.post0+cxx11.abi available.


In [16]:
def format_prompts(batch):
    formatted_prompts = []
    for instruction, response in zip(batch["instruction"], batch["response"]):
        prompt = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
        formatted_prompts.append(prompt)
    return {"text": formatted_prompts}

dataset = dataset.map(format_prompts, batched=True)
split_dataset = dataset.train_test_split(test_size=0.2, seed=99)
train_dataset = split_dataset["train"]
validation_dataset = split_dataset["test"]

___
#### Step 7: Finetuning the Model 🏋️‍♂️

In [17]:
import transformers
import wandb

from trl import SFTTrainer

os.environ["WANDB_PROJECT"] = "falcon7b" # add wandb project name 
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["IPEX_TILE_AS_DEVICE"] = "1"

finetuned_model_id = "eduardo-alvarez/falcon-7B-dolly-sft-LoRA" # change to your info
PUSH_TO_HUB = True
USE_WANDB = True

# Calculate max_steps based on the subset size
num_train_samples = len(train_dataset)
batch_size = 2
gradient_accumulation_steps = 4
steps_per_epoch = num_train_samples // (batch_size * gradient_accumulation_steps)
num_epochs = 1
max_steps = steps_per_epoch * num_epochs
print(f"Finetuning for max number of steps: {max_steps}")

def print_training_summary(results):
    print(f"Time: {results.metrics['train_runtime']: .2f}")
    print(f"Samples/second: {results.metrics['train_samples_per_second']: .2f}")
    get_memory_usage()

training_args = transformers.TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_ratio=0.05,
        max_steps=max_steps,
        learning_rate=1e-5,
        evaluation_strategy="steps",
        save_steps=500,
        bf16=True,
        logging_steps=100,
        output_dir=finetuned_model_id,
        hub_model_id=finetuned_model_id if PUSH_TO_HUB else None,
        use_ipex=True,
        report_to="wandb" if USE_WANDB else None,
        push_to_hub=PUSH_TO_HUB,
        max_grad_norm=0.6,
        weight_decay=0.01,
        group_by_length=True
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    packing=True
)

if device != "cpu":
    print_memory_usage()
    torch.xpu.empty_cache()
results = trainer.train()
print_training_summary(results)
#wandb.finish()

# save lora model
tuned_lora_model = "falcon-7B-dolly-sft-LoRA" # change to your info
trainer.model.save_pretrained(tuned_lora_model)

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2300 > 2048). Running this sequence through the model will result in indexing errors


Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
2024-06-02 20:12:38,493 - wandb.jupyter - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
ERROR: ld.so: object '/opt/intel/oneapi/intelpython/lib/libtcmalloc.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
wandb: Currently logged in as: eduand-alvarez. Use `wandb login --relogin` to force relogin
ERROR: ld.so: object '/opt/intel/oneapi/intelpython/lib/libtcmalloc.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/opt/intel/oneapi/intelpython/lib/libtcmalloc.so' from LD_PRELOAD cannot be preloaded (cannot open shar

Step,Training Loss,Validation Loss
100,2.173100,2.116726
200,2.129000,2.066212
300,2.062200,1.986853
400,1.977700,1.942249
500,1.968300,1.938298


ERROR: ld.so: object '/opt/intel/oneapi/intelpython/lib/libtcmalloc.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
wandb: Adding directory to artifact (./eduardo-alvarez/falcon-7B-dolly-sft-LoRA/checkpoint-500)... Done. 0.2s
ERROR: ld.so: object '/opt/intel/oneapi/intelpython/lib/libtcmalloc.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
max_steps is given, it will override any value given in num_train_epochs


___
#### Step 8: Save the Finetuned Model 💾

In [18]:
from peft import PeftModel

tuned_model = "falcon-7B-dolly-sft-LoRA" # change to your info

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
)

model = PeftModel.from_pretrained(base_model, tuned_lora_model)
model = model.merge_and_unload()
# save final tuned model
model.save_pretrained(tuned_model)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

___

#### Disclaimer for Using Large Language Models

Please be aware that while Large Language Models like Camel-5B and OpenLLaMA 3b v2 are powerful tools for text generation, they may sometimes produce results that are unexpected, biased, or inconsistent with the given prompt. It's advisable to carefully review the generated text and consider the context and application in which you are using these models.

Usage of these models must also adhere to the licensing agreements and be in accordance with ethical guidelines and best practices for AI. If you have any concerns or encounter issues with the models, please refer to the respective model cards and documentation provided in the links above.

To the extent that any public or non-Intel datasets or models are referenced by or accessed using these materials those datasets or models are provided by the third party indicated as the content source. Intel does not create the content and does not warrant its accuracy or quality. By accessing the public content, or using materials trained on or with such content, you agree to the terms associated with that content and that your use complies with the applicable license.

Intel expressly disclaims the accuracy, adequacy, or completeness of any such public content, and is not liable for any errors, omissions, or defects in the content, or for any reliance on the content. Intel is not liable for any liability or damages relating to your use of public content.

Intel’s provision of these resources does not expand or otherwise alter Intel’s applicable published warranties or warranty disclaimers for Intel products or solutions, and no additional obligations, indemnifications, or liabilities arise from Intel providing such resources. Intel reserves the right, without notice, to make corrections, enhancements, improvements, and other changes to its materials.